### **1. (S) SINGLE RESPONSIBILITY PRINCIPLE**

_**Try to make every class responsible for a single part of the functionality provided by the software, and make that responsibility entirely encapsulated by (you can also say hidden within) the class.**_

#### **BEFORE**

In [55]:
class Employee:
    def __init__(self, name: str, salary: float) -> None:
        self.name = name
        self.salary = salary

    # The format of the timesheet report may change over time,
    # requiring you to change the code within the class.
    def time_sheet_report(self) -> str:
        return f"Timesheet report for {self.name!s}"

#### **AFTER**

In [56]:
class Employee:
    def __init__(self, name: str, salary: float) -> None:
        self.name = name
        self.salary = salary


# Solve the problem by moving the behavior related to printing
# timesheet reports into a separate class. This change lets you
# move other report-related stuff to the new class.
class TimeSheetReport:
    @staticmethod
    def print(employee: Employee) -> str:
        return f"Timesheet report for {employee.name!s}"

### **2. (O) OPEN/CLOSED PRINCIPLE**

_**Classes should be open for extension but closed for modification.**_ 

A class is open if you can extend it, produce a subclass and
do whatever you want with it—add new methods or fields,
override base behavior, etc. At the same time, the class is closed 
(you can also say complete) if it’s 100% ready to be used by other 
classes—its interface is clearly defined and won’t be changed in the future.

#### **BEFORE**

In [ ]:
from typing import Literal


# You have an e-commerce application with an Order class that
# calculates shipping costs and all shipping methods are hardcoded
# inside the class. If you need to add a new shipping
# method, you have to change the code of the Order class and
# risk breaking it.
class Order:
    def __init__(self, total: float, shipping: Literal["Air", "Ground"]) -> None:
        self.total = total
        self.shipping = shipping

    def shipping_cost(self) -> float:
        if self.shipping == "Ground":
            return self.total + 5  # $5 is the base cost for ground shipping
        if self.shipping == "Air":
            return self.total + 10  # $10 is the base cost for air shipping
        raise ValueError(f"Invalid shipping option {self.shipping!r}")

#### **AFTER**

In [58]:
from __future__ import annotations

from typing import Protocol


# You can solve the problem by applying the Strategy pattern. Start
# by extracting shipping methods into separate classes with a common
# interface.
class Shipping(Protocol):
    def get_cost(self, order: Order) -> float: ...


class GroundShipping:
    def get_cost(self, order: Order) -> float:
        return order.total + 5


class AirShipping:
    def get_cost(self, order: Order) -> float:
        return order.total + 10


class Order:
    def __init__(self, total: float, shipping: Shipping) -> None:
        self.total = total
        self.shipping = shipping

    def shipping_cost(self) -> float:
        return self.shipping.get_cost(self)

### **3. (L) LISKOV SUBSTITUTION PRINCIPLE**

_**When extending a class, remember that you should be able to pass objects of the subclass in place of objects of the parent class without breaking the client code.**_

This means that the subclass should remain compatible with
the behavior of the superclass. When overriding a method,
extend the base behavior rather than replacing it with something
else entirely. The substitution principle is a set of checks that help predict
whether a subclass remains compatible with the code that
was able to work with objects of the superclass. This concept
is critical when developing libraries and frameworks because
your classes are going to be used by other people whose code
you can’t directly access and change.

_**- Parameter types in a method of a subclass should match or be more abstract than parameter types in the method of the superclass.**_

In [70]:
class Animal: ...


class Cat(Animal): ...


class Meat:
    def feed(self, cat: Cat) -> None: ...


class Sausage(Meat):
    def feed(self, animal: Animal) -> None: ...

_**- The return type in a method of a subclass should match or be a subtype of the return type in the method of the superclass.**_

In [71]:
class Cat: ...


class BengalCat(Cat): ...


class CatShop:
    def buy_cat(self) -> Cat: ...


class BengalShop(CatShop):
    def buy_cat(self) -> BengalCat: ...

_**- A method in a subclass shouldn’t throw types of exceptions which the base method isn’t expected to throw.**_

In other words,
types of exceptions should match or be subtypes of the ones
that the base method is already able to throw.

_**- A subclass shouldn’t strengthen pre-conditions.**_

For example,
the base method has a parameter with type `int` . If a subclass
overrides this method and requires that the value of an
argument passed to the method should be positive (by throwing
an exception if the value is negative), this strengthens the
pre-conditions.

_**- A subclass shouldn’t weaken post-conditions.**_

Say you have a
class with a method that works with a database. A method of
the class is supposed to always close all opened database connections
upon returning a value.

_**- A subclass shouldn’t change values of private fields of the superclass.**_

### **4. (I) INTERFACE SEGREGATION PRINCIPLE**

_**- Clients shouldn’t be forced to depend on methods they do not use.**_

Try to make your interfaces narrow enough that client classes
don’t have to implement behaviors they don’t need. According to the interface segregation principle, you should
break down “fat” interfaces into more granular and specific
ones. Clients should implement only those methods that they
really need.

#### **BEFORE**

In [72]:
from typing import Protocol


class CloudProvider(Protocol):
    def store_file(self, name: str) -> None: ...
    def get_file(self, name: str) -> None: ...
    def create_server(self, region: str) -> None: ...
    def list_servers(self) -> None: ...


class AWSProvider(CloudProvider):
    def store_file(self, name: str) -> None:
        print(f"Storing {name!r} in S3")

    def get_file(self, name: str) -> None:
        print(f"Getting {name!r} from S3")

    def create_server(self, region: str) -> None:
        print(f"Creating server in {region!r}")

    def list_servers(self) -> None:
        print("Listing servers in AWS")


# DropboxProvider doesn't need the create_server and list_servers methods.
class DropboxProvider(CloudProvider):
    def store_file(self, name: str) -> None:
        print(f"Storing {name!r} in Dropbox")

    def get_file(self, name: str) -> None:
        print(f"Getting {name!r} from Dropbox")

#### **AFTER**

In [74]:
from typing import Protocol


class CloudHostingProvider(Protocol):
    def create_server(self, region: str) -> None: ...
    def list_servers(self) -> None: ...


class CloudStorageProvider(Protocol):
    def store_file(self, name: str) -> None: ...
    def get_file(self, name: str) -> None: ...


class AWSProvider(CloudHostingProvider, CloudStorageProvider):
    def store_file(self, name: str) -> None:
        print(f"Storing {name!r} in S3")

    def get_file(self, name: str) -> None:
        print(f"Getting {name!r} from S3")

    def create_server(self, region: str) -> None:
        print(f"Creating server in {region!r}")

    def list_servers(self) -> None:
        print("Listing servers in AWS")


# DropboxProvider implements only the CloudStorageProvider interface.
class DropboxProvider(CloudStorageProvider):
    def store_file(self, name: str) -> None:
        print(f"Storing {name!r} in Dropbox")

    def get_file(self, name: str) -> None:
        print(f"Getting {name!r} from Dropbox")

### **5. (D) DEPENDENCY INVERSION PRINCIPLE**

_**High-level classes shouldn’t depend on low-level classes. Both should depend on abstractions. Abstractions shouldn’t depend on details. Details should depend on abstractions.**_

Usually when designing software, you can make a distinction between two levels of classes:

- Low-level classes implement basic operations such as working with a disk, transferring data over a network, connecting to a database, etc.
- High-level classes contain complex business logic that directs low-level classes to do something.

Sometimes people design low-level classes first and only then
start working on high-level ones. This is very common when
you start developing a prototype on a new system, and you’re
not even sure what’s possible at the higher level because
low-level stuff isn’t yet implemented or clear. With such an
approach business logic classes tend to become dependent on
primitive low-level classes.

**The dependency inversion principle suggests changing the direction of this dependency.**